In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os, sys, itertools


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from util.params import ENRICH_ALPHA, MULTI_HYP_CORR_METHOD
pd.options.display.max_columns = 100

In [2]:
# exp_muts_df = pd.read_pickle("./data/9_df.pkl")  # Currently skipping imodulons for the sake of the AVA MS
exp_muts_df = pd.read_pickle("./data/8_df.pkl")
display(exp_muts_df.shape, exp_muts_df.head())

(148, 45)

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,pathways,pathway links,COGs,COG links,temperature,carbon-source,supplement,strain-description,taxonomy-id,base-media,nitrogen-source,phosphorous-source,sulfur-source,calcium-source
630,66,T247M (ACG→ATG),yafD,SNP,231861,NC_000913,C→T,1,GYD,280,0,0.72,1,True,"(231861, 231861)",{ECK120001594},"[{'name': 'yafD', 'RegulonDB ID': 'ECK12000159...",False,False,{},{},{},{},{},True,"[{'name': 'yafD', 'RegulonDB ID': 'ECK12000159...",{'ECK120001594': ['ECK120001594']},"[{'name': 'yafDE', 'RegulonDB ID': 'ECK1200291...",{'ECK120029169': ['ECK120001594']},[],{},[],{},[{'name': 'General function prediction only'}],{'General function prediction only': ['ECK1200...,37 celsius,glycerol(2.5),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
631,126,R665S (CGT→AGT),cyaA,SNP,3993145,NC_000913,C→A,1,GYD,280,0,1.00,1,True,"(3993145, 3993145)",{ECK120000166},"[{'name': 'cyaA', 'RegulonDB ID': 'ECK12000016...",False,False,{},{},{},{},{},True,"[{'name': 'cyaA', 'RegulonDB ID': 'ECK12000016...",{'ECK120000166': ['ECK120000166']},"[{'name': 'cyaA', 'RegulonDB ID': 'ECK12001483...",{'ECK120014835': ['ECK120000166']},"[{'name': 'CRP', 'significantly associated con...",{'CRP': ['cyaA']},[{'name': 'Purine metabolism'}],{'Purine metabolism': ['ECK120014835']},[{'name': 'Nucleotide transport and metabolism'}],{'Nucleotide transport and metabolism': ['ECK1...,37 celsius,glycerol(2.5),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
632,94,S59Y (TCC→TAC),glpK,SNP,4117047,NC_000913,G→T,1,GYD,280,0,1.00,1,True,"(4117047, 4117047)",{ECK120000391},"[{'name': 'glpK', 'RegulonDB ID': 'ECK12000039...",False,False,{},{},{},{},{},True,"[{'name': 'glpK', 'RegulonDB ID': 'ECK12000039...",{'ECK120000391': ['ECK120000391']},"[{'name': 'glpFKX', 'RegulonDB ID': 'ECK120014...",{'ECK120014723': ['ECK120000391']},"[{'name': 'GlpR', 'significantly associated co...","{'GlpR': ['glpFKX'], 'CRP': ['glpFKX']}",[{'name': 'Glycerolipid metabolism'}],{'Glycerolipid metabolism': ['ECK120014723']},[{'name': 'Energy production and conversion'}],{'Energy production and conversion': ['ECK1200...,37 celsius,glycerol(2.5),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
633,72,T593P (ACC→CCC),metL,SNP,4131611,NC_000913,A→C,1,GYD,280,0,0.67,1,True,"(4131611, 4131611)",{ECK120000583},"[{'name': 'metL', 'RegulonDB ID': 'ECK12000058...",False,False,{},{},{},{},{},True,"[{'name': 'metL', 'RegulonDB ID': 'ECK12000058...",{'ECK120000583': ['ECK120000583']},"[{'name': 'metBL', 'RegulonDB ID': 'ECK1200150...",{'ECK120015057': ['ECK120000583']},"[{'name': 'MetJ', 'significantly associated co...","{'MetJ': ['metBL'], 'PhoP': ['metBL'], 'HypT':...","[{'name': 'Glycine, serine and threonine metab...","{'Glycine, serine and threonine metabolism': [...",[{'name': 'Amino acid transport and metabolism'}],{'Amino acid transport and metabolism': ['ECK1...,37 celsius,glycerol(2.5),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
634,107,E641K (GAA→AAA),rpoB,SNP,4183165,NC_000913,G→A,1,GYD,280,0,1.00,1,True,"(4183165, 4183165)",{ECK120000885},"[{'name': 'rpoB', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoB', 'RegulonDB ID': 'ECK12000088...",{'ECK120000885': ['ECK120000885']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000885']},[],{},"[{'name': 'Purine metabolism'}, {'name': 'Pyri...","{'Purine metabolism': ['ECK120016992'], 'Pyrim...",[{'name': 'Transcription'}],{'Transcription': ['ECK120016992']},37 celsius,glycerol(2.5)

# significance

In [3]:
# a bit behind annotations currently used in AVA MS.
# Still uses same approach to counted, just not all data-types have been consolidated into the COGs feat col


comp_mut_df = pd.DataFrame(columns=["observed mutation count"])
for i, r in exp_muts_df.iterrows():
    for f, links in r["COG links"].items():
        if "unknown" not in f:
            if f not in comp_mut_df.index:
                df = pd.DataFrame({"observed mutation count": len(links)}, index=[f])
                comp_mut_df = comp_mut_df.append(df)
            else:
                comp_mut_df.loc[f, "observed mutation count"] += len(links)
            
comp_mut_df

,observed mutation count
General function prediction only,5
Nucleotide transport and metabolism,23
Energy production and conversion,39
Amino acid transport and metabolism,76
Transcription,27
Carbohydrate transport and metabolism,31
"Replication, recombination and repair",3
"Posttranslational modification, protein turnover, chaperones",1
Inorganic ion transport and metabolism,1
Coenzyme transport and metabolism,3


In [4]:
np.sum(comp_mut_df["observed mutation count"])

282

In [5]:
for _, r in exp_muts_df.iterrows():
    for feat_d in r["COGs"]:
        if "unknown" not in feat_d["name"]:
            feat_d["mutation set count"] = comp_mut_df.loc[feat_d["name"]]["observed mutation count"]

In [6]:
COG_len_df = pd.read_pickle("./data/COG_len_df.pkl")
COG_len_df.head()

,COG length
COG description,
Amino acid transport and metabolism,660007
Inorganic ion transport and metabolism,305849
Carbohydrate transport and metabolism,679809
Coenzyme transport and metabolism,212728
Energy production and conversion,589566


In [7]:
import numpy as np


comp_mut_df["length"] = comp_mut_df.apply(lambda r: COG_len_df.loc[r.index, "COG length"])
total_seq_len = np.sum(comp_mut_df["length"])
comp_mut_df["proportion"] = comp_mut_df["length"].apply(lambda comp_len: comp_len/total_seq_len)
comp_mut_df

,observed mutation count,length,proportion
General function prediction only,5,254764,0.044958
Nucleotide transport and metabolism,23,160339,0.028295
Energy production and conversion,39,589566,0.104041
Amino acid transport and metabolism,76,660007,0.116472
Transcription,27,424603,0.074930
Carbohydrate transport and metabolism,31,679809,0.119966
"Replication, recombination and repair",3,309957,0.054698
"Posttranslational modification, protein turnover, chaperones",1,270912,0.047808
Inorganic ion transport and metabolism,1,305849,0.053973
Coenzyme transport and metabolism,3,212728,0.037540


In [8]:
from collections import Counter
import numpy as np
comp_mut_df['equal or larger count'] = 0
component_l = comp_mut_df.index.tolist()
proportion_l = comp_mut_df["proportion"].tolist()
num_muts = np.sum(comp_mut_df["observed mutation count"])

# slides from UW speak about the relationship of permutations and precision.
# should read through them to get a sense of what is the least iterations that should execute.
num_itr = 10000
for i in range(0, num_itr):
    random_mut_comp_a = np.random.choice(
        a=component_l,
        size=int(num_muts),
        p=proportion_l,
        replace=True,
    )
    random_mut_comp_count_d = Counter(random_mut_comp_a)
    for component in random_mut_comp_count_d.keys():
        if random_mut_comp_count_d[component] >= comp_mut_df.loc[component, 'observed mutation count']:
            comp_mut_df.loc[component, 'equal or larger count'] += 1

comp_mut_df["p value"] = comp_mut_df["equal or larger count"].apply(
    lambda count: count/num_itr)

comp_mut_df["significant"] = comp_mut_df["p value"].apply(lambda p_val: True if p_val < ENRICH_ALPHA else False)

display(comp_mut_df.head())

,observed mutation count,length,proportion,equal or larger count,p value,significant
General function prediction only,5,254764,0.044958,9968,0.9968,False
Nucleotide transport and metabolism,23,160339,0.028295,0,0.0000,True
Energy production and conversion,39,589566,0.104041,392,0.0392,True
Amino acid transport and metabolism,76,660007,0.116472,0,0.0000,True
Transcription,27,424603,0.074930,1170,0.1170,False


In [9]:
from statsmodels.stats import multitest
pvals = comp_mut_df["p value"]
rejects, pvals_corrected, alphacSidak, alphacBonf = multitest.multipletests(
    pvals=pvals,
    alpha=ENRICH_ALPHA,
    method=MULTI_HYP_CORR_METHOD)

pd.options.display.max_rows = 500
comp_mut_df["corrected p value"] = pvals_corrected
comp_mut_df["corrected significance"] = rejects
comp_mut_df.head()

,observed mutation count,length,proportion,equal or larger count,p value,significant,corrected p value,corrected significance
General function prediction only,5,254764,0.044958,9968,0.9968,False,1.0000,False
Nucleotide transport and metabolism,23,160339,0.028295,0,0.0000,True,0.0000,True
Energy production and conversion,39,589566,0.104041,392,0.0392,True,0.7056,False
Amino acid transport and metabolism,76,660007,0.116472,0,0.0000,True,0.0000,True
Transcription,27,424603,0.074930,1170,0.1170,False,1.0000,False


In [10]:
# # not wanting to consider mutation events with only 1 features as being significant
comp_mut_df["final significance"] = comp_mut_df.apply(lambda r: r["corrected significance"] if r["observed mutation count"] > 1 else False, axis=1)
# comp_mut_df.head()

In [11]:
for i, r in exp_muts_df.iterrows():
    for d in r["COGs"]:
        if d["name"] in comp_mut_df.index:
            d["significantly enriched"] = comp_mut_df.loc[d["name"]]["final significance"]

In [12]:
assert(len(exp_muts_df.exp.unique())==1)
exp = exp_muts_df.exp.unique()[0]
comp_mut_df.to_csv("./data/supp/permutation_test_results_per_experiment/"+exp+"_mut_COG_signif.csv")

In [13]:
exp_muts_df.to_pickle("./data/10_df.pkl")

In [14]:
# import seaborn as sns
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.rcParams["figure.dpi"] = 100
# sns.set_context("talk")
# sns.set_style("ticks")


# def _get_name_signif_from_name(name, json):
#     is_signif = False
#     for d in json:
#         if d["name"] == name:
#             is_signif = d["significant"]
#             break
#     return is_signif


# # The reason we're using the links dictionary is because we want to count how genomic features were mutated for the genetic target.
# targ_cnt_d = dict()
# for _, r in exp_muts_df.iterrows():
#     link_d = r["COG links"]
#     for targ_name, hits in link_d.items():
#         is_signif = _get_name_signif_from_name(targ_name, r["COGs"])
#         t = str(targ_name)
#         if is_signif:
#             t = '*' + t
#         if t not in targ_cnt_d.keys():
#             targ_cnt_d[t] = 0
#         targ_cnt_d[t] += len(hits)
        
# df = pd.DataFrame.from_dict(targ_cnt_d, orient='index', columns=["mutated feature count"])
# df = df.sort_values("mutated feature count")
# df = df[-20:]
# ax = df.plot.barh(y='mutated feature count',
# #                   figsize=(10,25),
#                   figsize=(5,10),
#                   color="#4C72B0",
#                   width=1
#                  )
# for i, v in enumerate(df["mutated feature count"]):
#     ax.text(v, i - 0.2, str(v))
# sns.despine(ax=ax, top=True, right=True)
# ax.set_title("Top 20 mutated COGs")
# ax.get_legend().remove()

In [15]:
# # The reason we're using the links dictionary is because we want to count how genomic features were mutated for the genetic target.
# targ_cnt_d = dict()
# for _, r in exp_muts_df.iterrows():
#     link_d = r["COG links"]
#     for targ_name, hits in link_d.items():
#         is_signif = _get_name_signif_from_name(targ_name, r["COGs"])
#         t = str(targ_name)
#         if is_signif:
#             t = '*' + t
#             if t not in targ_cnt_d.keys():
#                 targ_cnt_d[t] = 0
#             targ_cnt_d[t] += len(hits)
# if len(targ_cnt_d) > 0:
#     df = pd.DataFrame.from_dict(targ_cnt_d, orient='index', columns=["mutated feature count"])
#     df = df.sort_values("mutated feature count")
#     df = df[-20:]
#     ax = df.plot.barh(y='mutated feature count',
#     #                   figsize=(10,25),
#     #                   figsize=(5,10),
#                       color="#4C72B0",
#                       width=1
#                      )
#     for i, v in enumerate(df["mutated feature count"]):
#         ax.text(v, i - 0.2, str(v))
#     sns.despine(ax=ax, top=True, right=True)
#     ax.set_title("Signifcant mutated COGs")
#     ax.get_legend().remove()